In [642]:
from tweepy import API
from tweepy import Cursor
#from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import RateLimitError
import pandas as pd
import re


In [643]:
consumer_token='2qhjD55Se9NGKRTBuZyb6Ki0Z'
consumer_secret= 'KooojZyWYxp7iu6gerfpwyJB1xbOVZHxRXYcO8L7VsF0CqQPAt'
access_token='352599159-PO9igRuIq6bWZgYcLU9AAeONtQarHyswa9SsW3hr'
access_token_secret='qhPSxUDrBIq8AQu4nTeiXBlkC5rthIlFcqGOYfOEvkGrX'




In [644]:
#setting up authorization
auth = OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
# Creation of the actual interface, using authentication
api = API(auth)
api.wait_on_rate_limit = True
 
# Sample method, used to update a status
#api.update_status('Hello Python Central!')

In [645]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [660]:
build_aggregation_df(booker_df)

,counts,pcts,codes,text
AK,2,0.58,AK,Count: 2 Percentage: 0.58
AL,5,1.46,AL,Count: 5 Percentage: 1.46
AR,2,0.58,AR,Count: 2 Percentage: 0.58
AZ,7,2.05,AZ,Count: 7 Percentage: 2.05
CA,40,11.70,CA,Count: 40 Percentage: 11.7
CO,6,1.75,CO,Count: 6 Percentage: 1.75
CT,6,1.75,CT,Count: 6 Percentage: 1.75
FL,20,5.85,FL,Count: 20 Percentage: 5.85
GA,10,2.92,GA,Count: 10 Percentage: 2.92
IA,1,0.29,IA,Count: 1 Percentage: 0.29


In [646]:
#setup
acceptable_locs = {
    'AL':['al','alabama'],
    'AK':['ak','alaska'], 
    'AZ':['az','arizona'], 
    'AR':['ar','arkansas'], 
    'CA':['ca','california'], 
    'CO':['co','colorado'], 
    'CT':['ct','connecticut'], 
    'DE':['de','delaware'], 
    'FL':['fl','florida'], 
    'GA':['ga','georgia'], 
    'HI':['hi','hawaii'], 
    'ID':['id','idaho'], 
    'IL':['il','illinois'], 
    'IN':['in','indiana'], 
    'IA':['ia','iowa'], 
    'KS':['ks','kansas'], 
    'KY':['ky','kentucky'], 
    'LA':['la','louisiana'], 
    'ME':['me','maine'], 
    'MD':['md','maryland'], 
    'MA':['ma','massachusetts'], 
    'MI':['mi','michigan'], 
    'MN':['mn','minnesota'], 
    'MS':['ms','mississippi'], 
    'MO':['mo','missouri'], 
    'MT':['mt','montana'],
    'NE':['ne','nebraska'], 
    'NV':['nv','nevada'], 
    'NH':['nh','newhampshire'], 
    'NJ':['nj','newjersey','jersey'], 
    'NM':['nm','newmexico'], 
    'NY':['ny','newyork','nyc'],
    'NC':['nc','northcarolina'],
    'ND':['nd','northdakota'],
    'OH':['oh','ohio'],
    'OK':['ok','oklahoma'],
    'OR':['or','oregon'],
    'PA':['pa','pennsylvania'],
    'RI':['ri','rhodeisland','rhode'],
    'SC':['sc','southcarolina'],
    'SD':['sd','southdakota'],
    'TN':['tn','tennessee'],
    'TX':['tx','texas'],
    'UT':['ut','utah'],
    'VT':['vt','vermont'],
    'VA':['va','virginia'],
    'WA':['wa','washington'],
    'WV':['wv','westvirginia'],
    'WI':['wi','wisconsin'],
    'WY':['wy','wyoming'],
}

#unpacking/melting above dictionary to list for direct access
acceptable_terms = []
for each in acceptable_locs.values():
    acceptable_terms.extend(each)

In [647]:
def build_df_from_api(screen_name):
    #return n=10 tweets + column of lists of retweeter ids
    df = get_n_tweets_df(screen_name)
    #returns user defined locations of retweeters
    df = add_retweet_geos(df)
    #add column of cleaned up locations
    df['parsed_locs'] = df['retweeter_geos'].map(lambda x: locs_parser(x))
    #add column of location count as dictionary 
    df['rt_loc_counts'] = df['parsed_locs'].map(lambda x: item_counter(x))

    return df

def build_aggregation_df(df):
    #collapse rows of dictionaries to summation series where each row is a dictionary key
    aggdic_srs = dict_srs_agg(df['rt_loc_counts'])
    #print(aggdic_srs)
    #adds percentage column which coerces to df
    newdf = add_pcts_to_srs(aggdic_srs)
    #adds copy of index as column to facilitate working with plotly
    newdf['codes'] = newdf.index
    newdf['text'] = 'Count: '+newdf['counts'].astype(str)+' '+\
'Percentage: '+newdf['pcts'].astype(str)

    return newdf
    


In [649]:
kamala_df =build_df_from_api('KamalaHarris')

In [655]:
kamala_df.loc[4]['entities']

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/m5OtX6UFg1',
   'expanded_url': 'https://twitter.com/i/web/status/1095883581585584128',
   'display_url': 'twitter.com/i/web/status/1…',
   'indices': [117, 140]}]}

In [654]:
py.iplot(data_compiler(build_aggregation_df(kamala_df)),filename='kamala rts')

In [484]:
#funcs with api calls
def get_n_tweets_df(screen_name,n=10):
    wanted_status_keys = ['id','text','created_at','favorite_count','retweet_count','retweeter_ids','entities']
    temp = pd.DataFrame([tweet._json for tweet in Cursor(api.user_timeline, screen_name, include_rts=False).items(n)])
    temp['retweeter_ids'] = temp['id'].map(lambda x:api.retweeters(x))
    return temp[wanted_status_keys]
    
def get_location_by_id(idnum):
    return api.get_user(idnum)._json['location']

def add_retweet_geos(df):
    df['retweeter_geos'] = df['retweeter_ids'].map(lambda x: [get_location_by_id(idnum) for idnum in x])
    return df




In [598]:
#funcs with local scope

def state_name_collapser(string):
    """
    eliminates space after given keywords
    """
    string = re.sub(r'north[ ]','north',string)
    string = re.sub(r'south[ ]','south',string)
    string = re.sub(r'west[ ]','west',string)
    string = re.sub(r'new[ ]','new',string)
    return string

def get_real_location(loc):
    #normalizing input, seperating into list
    clean = state_name_collapser(loc.replace(',','').lower())
    clean_split = clean.split()
    
    #filtering for words that show up in our list of acceptable terms
    dropping_trash = [word for word in clean_split if word in acceptable_terms]
    
    #return key (ex 'NY' for 'new york') if associated value is found in dropping_trash
    output = ''
    for each,lst in acceptable_locs.items():
         for item in lst:
                if item in dropping_trash:
                    output += each
                    
    #HACK ALERT: if more than 1 associated key found (ex: 'california new york'-->'CANY', only returns 1 key (alphabetically)         
    return output[:2]

def locs_parser(lstoflocs):
    return list(filter(None,[get_real_location(name) for name in lstoflocs]))

def item_counter(lst):
    tempdic={}
    for each in lst:
        if each in tempdic:
            tempdic[each] += 1
        else:
            tempdic[each] = 1
    return tempdic

def dict_srs_agg(srs):
    tempdic= {}
    for dct in srs:
       # print(dct)
        for k,v in dct.items():
           # print(k,v)
            if k in tempdic:
                tempdic[k] += v
            else:
                tempdic[k] = v
    
    return pd.Series(tempdic)

def add_pcts_to_srs(srs):
    newdf = pd.DataFrame({'counts':srs})
    newdf['pcts'] = newdf['counts'].apply(lambda x: round(x/newdf['counts'].sum()*100,2))
    return newdf.sort_index()


    

            
        

    

In [591]:
booker_rt_cts

TX    18
NY    29
NJ    62
NH     1
IL     9
OR    11
AL     5
CA    40
GA    10
VT     1
NC     8
WA    21
AZ     7
IN     9
MA     9
PA     5
NV     6
CO     6
OH     9
FL    20
KS     4
ME     3
OK     3
VA     5
AK     2
CT     6
SC     2
NM     3
KY     3
MI     5
WV     1
LA     6
AR     2
IA     1
MO     1
MN     2
TN     3
UT     1
WI     1
NE     1
MD     1
dtype: int64

In [610]:
# master_df = pd.concat([warren_rt_cts,booker_rt_cts],axis=1,sort=True)
# master_df.columns = ['warren','booker']
# master_df

In [611]:
%matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls

In [522]:
tls.set_credentials_file(username='colemiller94', api_key='sLd4OQ9PvgnKyFNkArdd')



In [653]:
def data_compiler(df):
    data = [dict(
        type='choropleth',
        autocolorscale = True,
        locations = df['codes'],
        z = df['counts'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2))
    )]

    layout = dict(
        title = 'RT Map',
        geo = dict(
            scope='usa',
            projection = dict(type = 'albers usa'),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'
        )
    )
    fig = dict(data = data, layout=layout)
    return fig
    

In [ ]:
data_compiler

In [638]:
py.iplot(data_compiler(build_aggregation_df(booker_df)), filename= 'rt_map')





In [567]:
py.iplot(fig, filenamme = 'booker_rt_map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~colemiller94/0 or inside your plot.ly account where it is named 'plot from API'


In [657]:
dfs = [booker_df,warren_df]

In [658]:
dfs[0]

,id,coordinates,text,created_at,favorite_count,retweet_count,rt_loc_counts
0,1095788647654273041,None,"When I think of how bright our future is, I th...",Wed Feb 13 20:55:47 +0000 2019,133,23,"{'TX': 1, 'NY': 3, 'NJ': 5, 'NH': 1, 'IL': 1, ..."
1,1095346807398416385,None,"Today, we honor @PrincipalAkbar, Principal of ...",Tue Feb 12 15:40:04 +0000 2019,384,64,"{'CA': 4, 'GA': 1, 'TX': 2, 'NJ': 8, 'VT': 1, ..."
2,1095046563238477824,None,Today for #BlackHistoryMonth we recognize Shei...,Mon Feb 11 19:47:00 +0000 2019,470,109,"{'NJ': 27, 'OH': 1, 'PA': 1, 'NY': 1, 'IN': 1,..."
3,1092812200148979713,None,Happy Lunar New Year to all who celebrate. Wis...,Tue Feb 05 15:48:26 +0000 2019,249,29,"{'NY': 1, 'OR': 1, 'IN': 2, 'NJ': 2, 'MA': 1, ..."
4,1092539856054636544,None,Proud that today New Jersey signed a bill to r...,Mon Feb 04 21:46:14 +0000 2019,733,144,"{'VA': 2, 'OR': 1, 'NY': 1, 'MA': 3, 'CA': 6, ..."
5,1092473012639723520,None,"A young man once wrote to Rosa Parks: ""I wonde...",Mon Feb 04 17:20:38 +0000 2019,381,101,"{'FL': 3, 'CA': 7, 'TX': 4, 'NY': 2, 'OR': 2, ..."
6,1088888135222730752,None,Finally – looks like President Trump is agreei...,Fri Jan 25 19:55:36 +0000 2019,4292,715,"{'NC': 1, 'NV': 1, 'NJ': 5, 'TX': 2, 'NY': 4, ..."
7,1088858249116504066,None,"""This is about our safety &amp; security &amp;...",Fri Jan 25 17:56:51 +0000 2019,1035,304,"{'CT': 1, 'NC': 3, 'FL': 4, 'NY': 4, 'OH': 1, ..."
8,1088841006798258176,None,Major airports in NJ / NY area are experiencin...,Fri Jan 25 16:48:20 +0000 2019,423,151,"{'NC': 1, 'NY': 5, 'NJ': 8, 'CA': 4, 'FL': 1, ..."
9,1085594202036207616,None,McConnell can end this #TrumpShutdown right no...,Wed Jan 16 17:46:41 +0000 2019,528,166,"{'NY': 8, 'NC': 1, 'NJ': 2, 'CA': 5, 'WA': 6, ..."
